In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Bahia - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_BA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Desemprego,Bahia - IDH Educacao,Bahia - IDH Renda,...,Taxa Selic (%),Bahia - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Bahia - Consumo de Cimento (t)
0,2008,1.396949e+08,8.691699e+06,8.687931,1.237178e+08,0.783758,0.685154,8.102744,0.620781,0.659869,...,11.760983,874.866956,19.314465,1008.202617,1303.749515,26.386473,23.042487,77.493797,1.586819,2678.337
1,2009,1.406176e+08,8.722094e+06,8.695651,1.237698e+08,0.785624,0.687239,8.129293,0.624755,0.660519,...,9.472585,1028.843987,20.202505,1007.958668,1414.729195,25.968454,22.350784,78.022515,1.421020,2825.491
2,2010,1.439300e+08,8.745548e+06,8.730895,1.244348e+08,0.786978,0.689024,8.095385,0.628561,0.660850,...,9.332485,1146.660535,20.155807,1008.053425,1424.927462,25.603573,21.719583,78.179391,1.422041,3349.109
3,2011,1.469771e+08,8.709535e+06,8.759313,1.249553e+08,0.787829,0.690520,8.084857,0.632198,0.660869,...,10.985689,1256.520719,20.048279,1008.445156,1444.966819,25.564945,21.470218,77.933380,1.514544,3615.893
4,2012,1.498366e+08,8.659814e+06,8.775083,1.254646e+08,0.788133,0.691525,13.555307,0.635421,0.660403,...,8.159014,1339.122148,20.344237,1008.651747,1446.757048,25.595626,21.547865,78.542856,1.555217,3744.322
5,2013,1.523819e+08,8.603518e+06,8.785884,1.258886e+08,0.787978,0.692653,14.373236,0.638956,0.659936,...,7.913561,1340.894132,20.252405,1008.935896,1429.407124,25.586976,21.584976,78.969598,1.570158,3932.905
6,2014,1.543977e+08,8.508098e+06,8.799997,1.261078e+08,0.787476,0.693707,15.102184,0.642702,0.659347,...,10.350442,1284.691022,17.807272,1009.223255,1433.335843,25.757283,21.442184,77.696073,1.560276,4055.140
7,2015,1.558374e+08,8.390263e+06,8.806885,1.261743e+08,0.786515,0.693682,15.644545,0.644677,0.657960,...,12.454398,1218.786326,17.456441,1009.318844,1452.882864,25.895216,21.251973,76.266812,1.538323,3732.552
8,2016,1.565535e+08,8.301516e+06,8.804784,1.260566e+08,0.796009,0.701870,15.852241,0.655705,0.662422,...,13.111908,1144.727017,19.032574,1009.311974,1455.457303,25.838885,21.111445,75.883665,1.523445,3424.968
9,2017,1.566919e+08,8.249496e+06,8.807487,1.259606e+08,0.795010,0.701585,15.631769,0.655678,0.662461,...,9.479812,1108.624755,20.787217,1009.248599,1454.646572,25.882215,21.182739,76.030495,1.479634,3063.300


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Desemprego,Bahia - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Bahia - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,1.396949e+08,8.691699e+06,8.687931,1.237178e+08,0.783758,0.685154,8.102744,0.620781,0.659869,5.483335e+08,...,2.868108,11.760983,874.866956,19.314465,1008.202617,1303.749515,26.386473,23.042487,77.493797,1.586819
1,1.406176e+08,8.722094e+06,8.695651,1.237698e+08,0.785624,0.687239,8.129293,0.624755,0.660519,5.302764e+08,...,2.305885,9.472585,1028.843987,20.202505,1007.958668,1414.729195,25.968454,22.350784,78.022515,1.421020
2,1.439300e+08,8.745548e+06,8.730895,1.244348e+08,0.786978,0.689024,8.095385,0.628561,0.660850,5.046425e+08,...,1.958246,9.332485,1146.660535,20.155807,1008.053425,1424.927462,25.603573,21.719583,78.179391,1.422041
3,1.469771e+08,8.709535e+06,8.759313,1.249553e+08,0.787829,0.690520,8.084857,0.632198,0.660869,4.854622e+08,...,2.079088,10.985689,1256.520719,20.048279,1008.445156,1444.966819,25.564945,21.470218,77.933380,1.514544
4,1.498366e+08,8.659814e+06,8.775083,1.254646e+08,0.788133,0.691525,13.555307,0.635421,0.660403,4.867661e+08,...,2.183282,8.159014,1339.122148,20.344237,1008.651747,1446.757048,25.595626,21.547865,78.542856,1.555217
5,1.523819e+08,8.603518e+06,8.785884,1.258886e+08,0.787978,0.692653,14.373236,0.638956,0.659936,5.225848e+08,...,2.403445,7.913561,1340.894132,20.252405,1008.935896,1429.407124,25.586976,21.584976,78.969598,1.570158
6,1.543977e+08,8.508098e+06,8.799997,1.261078e+08,0.787476,0.693707,15.102184,0.642702,0.659347,5.676413e+08,...,2.748397,10.350442,1284.691022,17.807272,1009.223255,1433.335843,25.757283,21.442184,77.696073,1.560276
7,1.558374e+08,8.390263e+06,8.806885,1.261743e+08,0.786515,0.693682,15.644545,0.644677,0.657960,5.179392e+08,...,2.501529,12.454398,1218.786326,17.456441,1009.318844,1452.882864,25.895216,21.251973,76.266812,1.538323
8,1.565535e+08,8.301516e+06,8.804784,1.260566e+08,0.796009,0.701870,15.852241,0.655705,0.662422,4.524144e+08,...,2.073464,13.111908,1144.727017,19.032574,1009.311974,1455.457303,25.838885,21.111445,75.883665,1.523445
9,1.566919e+08,8.249496e+06,8.807487,1.259606e+08,0.795010,0.701585,15.631769,0.655678,0.662461,4.213689e+08,...,2.177386,9.479812,1108.624755,20.787217,1009.248599,1454.646572,25.882215,21.182739,76.030495,1.479634


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     2825.491
1     3349.109
2     3615.893
3     3744.322
4     3932.905
5     4055.140
6     3732.552
7     3424.968
8     3063.300
9     2990.152
10    3245.585
11    3803.900
12    3862.535
13    3948.984
14         NaN
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Desemprego,Bahia - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Bahia - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.801595,0.936957,-1.987321,-1.867929,-1.460829,-1.587856,-1.391013,-1.635715,-0.598881,1.204752,...,2.026352,0.984445,-2.321104,-0.494083,-1.206958,-3.089913,2.478541,2.695570,0.052337,1.398792
1,-1.650542,1.089552,-1.802089,-1.807175,-0.960333,-1.212869,-1.383171,-1.312206,-0.056326,0.820657,...,0.085558,-0.051710,-1.111409,0.005518,-1.706936,-0.276913,0.615792,1.366860,0.603496,-1.378567
2,-1.108275,1.207298,-0.956366,-1.029617,-0.597406,-0.891692,-1.393187,-1.002422,0.220138,0.275392,...,-1.114490,-0.115145,-0.185803,-0.020754,-1.512730,-0.018418,-1.010165,0.154369,0.767031,-1.361464
3,-0.609433,1.026498,-0.274462,-0.421068,-0.369250,-0.622546,-1.396297,-0.706399,0.236503,-0.132596,...,-0.697344,0.633403,0.677295,-0.081248,-0.709872,0.489519,-1.182293,-0.324643,0.510578,0.188100
4,-0.141312,0.776880,0.103974,0.174367,-0.287683,-0.441660,0.219728,-0.444119,-0.153228,-0.104860,...,-0.337667,-0.646476,1.326239,0.085254,-0.286461,0.534896,-1.045577,-0.175489,1.145924,0.869421
5,0.275387,0.494258,0.363149,0.670072,-0.329395,-0.238888,0.461353,-0.156393,-0.542565,0.657046,...,0.422337,-0.757614,1.340160,0.033591,0.295906,0.095128,-1.084122,-0.104202,1.590780,1.119708
6,0.605385,0.015214,0.701796,0.926438,-0.463954,-0.049203,0.676692,0.148528,-1.033902,1.615453,...,1.613108,0.345772,0.898610,-1.342011,0.884853,0.194709,-0.325214,-0.378496,0.263198,0.954156
7,0.841077,-0.576356,0.867096,1.004133,-0.721442,-0.053598,0.836911,0.309292,-2.192189,0.558230,...,0.760923,1.298413,0.380841,-1.539385,1.080764,0.690167,0.289436,-0.743877,-1.226730,0.586418
8,0.958309,-1.021897,0.816661,0.866585,1.824027,1.419319,0.898266,1.206866,1.532636,-0.835563,...,-0.716758,1.596124,-0.200994,-0.652671,1.066684,0.755421,0.038417,-1.013821,-1.626140,0.337201
9,0.980971,-1.283056,0.881520,0.754314,1.556137,1.368124,0.833136,1.204684,1.564684,-1.495939,...,-0.358019,-0.048438,-0.484625,0.334470,0.936796,0.734872,0.231503,-0.876871,-1.473078,-0.396703


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.71810187, -1.20524235,  0.39000188,  0.19491723,
          0.6699809 ,  1.03722788,  0.76732554,  1.09587231,
          0.46194009, -1.0453709 ,  1.21859064, -0.99976018,
          0.79428371,  1.15575749, -0.52559858,  1.30260559,
         -0.55253046, -0.25130359,  1.38787176,  1.47154451,
         -1.80649616,  1.09296291,  0.94281138,  0.52836084,
          0.61258103,  0.87656257,  1.47801536,  1.71982086,
          1.28221181,  1.39086472,  1.58209844,  1.08872091,
          0.55212962, -1.81398118,  1.44113674,  1.11191094,
          1.30470835,  0.86390516, -1.67365968, -1.70738939,
         -1.55813196, -0.48498301, -0.75714639,  0.40770893,
         -0.27394734,  2.02323219,  1.2908951 ,  1.70465513,
          1.81148084,  1.39968653, -1.36191442,  1.28024593,
          1.11054795,  0.12662799,  1.40414112,  1.40468999,
          1.45643959,  1.58509336,  1.32283157,  1.30222047,
          1.55138602,  1.91801602, -0.77489784,  2.32098884,
         -1.1942541 , -1

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     2825.491
1     3349.109
2     3615.893
3     3744.322
4     3932.905
5     4055.140
6     3732.552
7     3424.968
8     3063.300
9     2990.152
10    3245.585
11    3803.900
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    3803.9
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Desemprego,Bahia - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Bahia - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.416125,-1.086745,-0.129344,-0.207609,0.000408,0.737609,0.749286,0.972824,-0.47095,1.108405,...,-1.482779,-2.260447,0.290535,1.940009,0.112434,-1.032368,-0.187335,-0.1229,0.783824,-1.669544


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 4.16124690e-01, -1.08674459e+00, -1.29343935e-01,
         -2.07608869e-01,  4.07887568e-04,  7.37609122e-01,
          7.49285847e-01,  9.72823691e-01, -4.70950125e-01,
          1.10840491e+00,  2.63582309e+00, -1.89959107e-01,
          7.26404893e-01,  2.66833308e+00, -3.26320001e-02,
          9.88024351e-01,  2.49804942e-01, -8.95580792e-02,
          1.85434394e+00,  2.02706568e+00, -2.21150703e+00,
          1.42938591e+00,  8.51326681e-01,  1.40923863e-01,
          3.09023235e-01,  1.00267904e+00,  2.27970137e+00,
          1.11272593e+00,  6.10079019e-01,  8.90645378e-01,
          1.42231438e+00,  1.02650562e+00,  1.47127593e+00,
         -1.53201473e+00,  1.73503858e+00,  1.66610613e+00,
          1.04568940e+00,  5.52892590e-01, -1.81216593e+00,
         -2.10342296e+00, -2.55377478e+00,  4.19406277e-01,
         -1.75756445e+00,  2.41022998e-01, -9.86593447e-01,
          1.52184916e+00,  1.01398607e+00,  1.49290967e+00,
          1.40764092e+00,  1.87930411e+0

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    3862.535
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [18]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [20]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3796538544, 2649886956, 2873544498, 2913224235, 1849331431, 113498013, 2534156075, 669612623, 2663740089, 4264181536, 2079437449, 1006373976, 2632939653, 1037459314, 3592645304, 2152296851, 4139618489, 1993230630, 1717550809, 1703509903, 3948653691, 3456950489, 3940468088, 2574246025, 402525962, 1124203654, 4056598663, 2488727081, 3342083608, 2008630608, 1020751909, 2936843404, 3850328778, 1839561499, 2594774934, 1658510323, 2516948165, 2776900630, 2669110727, 35346596, 3138240454, 772027399, 3091108578, 2221598139, 148802398, 1586549490, 4115395233, 3366715787, 1500440446, 2297443404, 3884244410, 582626340, 1824770945, 3327826347, 191099657, 2632452418, 877534571, 3998429406, 782737986, 4097601333, 2027841604, 7315578, 205246332, 2165853148, 13526414, 3316440482, 251725992, 2980374031, 2517719689, 345939557, 1157178046, 481791600, 1178581328, 3570737599, 2869463587, 4071358324, 2584916300, 1885801505, 773736375, 1643524733, 1496709041, 3645944538, 3233963039, 3230296489, 4204979877, 

2023-09-08 14:56:15.322839: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-08 14:56:15.343386: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-08 14:56:15.343581: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

loss: 35988.12109375
winner_seed: 3796538544


Step: 1 ___________________________________________
loss: 24853.046875
winner_seed: 2649886956


Step: 2 ___________________________________________
loss: 25473.291015625


Step: 3 ___________________________________________
loss: 32369.353515625


Step: 4 ___________________________________________
loss: 16951.3046875
winner_seed: 1849331431


Step: 5 ___________________________________________
loss: 25726.0234375


Step: 6 ___________________________________________
loss: 28029.748046875


Step: 7 ___________________________________________
loss: 36207.74609375


Step: 8 ___________________________________________
loss: 12650.8173828125
winner_seed: 2663740089


Step: 9 ___________________________________________
loss: 21687.64453125


Step: 10 ___________________________________________
loss: 15944.484375


Step: 11 ___________________________________________
loss: 21060.912109375


Step: 12 ___________________________________________
l

In [21]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 12266481.0000
Epoch 2/10000
1/1 [==============================] - 0s 4ms/step - loss: 11755996.0000
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 4110935.2500
Epoch 4/10000
1/1 [==============================] - 0s 4ms/step - loss: 1828086.3750
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 244324.8125
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 132013.2031
Epoch 7/10000
1/1 [==============================] - 0s 4ms/step - loss: 167198.3281
Epoch 8/10000
1/1 [==============================] - 0s 4ms/step - loss: 178139.1250
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 95958.1484
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 209448.6875
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 205773.9219
Epoch 12/10000
1/1 [==============================] - 0s 4m

1/1 [==============================] - 0s 4ms/step - loss: 115253.7734
Epoch 97/10000
1/1 [==============================] - 0s 4ms/step - loss: 115503.2734
Epoch 98/10000
1/1 [==============================] - 0s 4ms/step - loss: 114164.7266
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 68738.9688
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 119828.7500
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 56066.5156
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 102515.1328
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 94461.3672
Epoch 104/10000
1/1 [==============================] - 0s 5ms/step - loss: 111245.6875
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 112252.2266
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 62342.9883
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step

1/1 [==============================] - 0s 4ms/step - loss: 102981.8516
Epoch 191/10000
1/1 [==============================] - 0s 4ms/step - loss: 63253.2539
Epoch 192/10000
1/1 [==============================] - 0s 4ms/step - loss: 72623.6641
Epoch 193/10000
1/1 [==============================] - 0s 5ms/step - loss: 120486.4062
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 53409.9883
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 94995.4062
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 117706.1562
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 91620.9688
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 134542.7969
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 88979.6328
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 127117.5547
Epoch 201/10000
1/1 [==============================] - 0s 4ms/ste

1/1 [==============================] - 0s 4ms/step - loss: 199425.0156
Epoch 285/10000
1/1 [==============================] - 0s 4ms/step - loss: 120736.3047
Epoch 286/10000
1/1 [==============================] - 0s 4ms/step - loss: 90289.9922
Epoch 287/10000
1/1 [==============================] - 0s 4ms/step - loss: 94149.0000
Epoch 288/10000
1/1 [==============================] - 0s 4ms/step - loss: 133787.8906
Epoch 289/10000
1/1 [==============================] - 0s 4ms/step - loss: 133071.8125
Epoch 290/10000
1/1 [==============================] - 0s 3ms/step - loss: 65827.6328
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 79710.5625
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 153369.8750
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 113856.3750
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 125768.0078
Epoch 295/10000
1/1 [==============================] - 0s 4ms/s

1/1 [==============================] - 0s 4ms/step - loss: 99263.7109
Epoch 379/10000
1/1 [==============================] - 0s 4ms/step - loss: 61088.2461
Epoch 380/10000
1/1 [==============================] - 0s 4ms/step - loss: 44877.5508
Epoch 381/10000
1/1 [==============================] - 0s 4ms/step - loss: 111862.4141
Epoch 382/10000
1/1 [==============================] - 0s 4ms/step - loss: 131431.2969
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 152529.1250
Epoch 384/10000
1/1 [==============================] - 0s 4ms/step - loss: 71269.4922
Epoch 385/10000
1/1 [==============================] - 0s 4ms/step - loss: 92393.2422
Epoch 386/10000
1/1 [==============================] - 0s 4ms/step - loss: 100116.6016
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 127983.4609
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 38095.4648
Epoch 389/10000
1/1 [==============================] - 0s 4ms/ste

...

1/1 [==============================] - 0s 4ms/step - loss: 80777.9922
Epoch 1136/10000
1/1 [==============================] - 0s 4ms/step - loss: 93151.0234
Epoch 1137/10000
1/1 [==============================] - 0s 4ms/step - loss: 77681.0859
Epoch 1138/10000
1/1 [==============================] - 0s 4ms/step - loss: 117263.0625
Epoch 1139/10000
1/1 [==============================] - 0s 4ms/step - loss: 104975.7500
Epoch 1140/10000
1/1 [==============================] - 0s 4ms/step - loss: 97043.6875
Epoch 1141/10000
1/1 [==============================] - 0s 4ms/step - loss: 165699.9375
Epoch 1142/10000
1/1 [==============================] - 0s 4ms/step - loss: 109286.8750
Epoch 1143/10000
1/1 [==============================] - 0s 4ms/step - loss: 100362.7422
Epoch 1144/10000
1/1 [==============================] - 0s 4ms/step - loss: 43597.5547
Epoch 1145/10000
1/1 [==============================] - 0s 4ms/step - loss: 130614.9609
Epoch 1146/10000
1/1 [==============================] 

1/1 [==============================] - 0s 4ms/step - loss: 101858.8438
Epoch 1230/10000
1/1 [==============================] - 0s 4ms/step - loss: 33658.9258
Epoch 1231/10000
1/1 [==============================] - 0s 4ms/step - loss: 53917.2461
Epoch 1232/10000
1/1 [==============================] - 0s 4ms/step - loss: 55888.7188
Epoch 1233/10000
1/1 [==============================] - 0s 4ms/step - loss: 50578.2930
Epoch 1234/10000
1/1 [==============================] - 0s 4ms/step - loss: 55124.7930
Epoch 1235/10000
1/1 [==============================] - 0s 4ms/step - loss: 59646.6211
Epoch 1236/10000
1/1 [==============================] - 0s 4ms/step - loss: 108244.1172
Epoch 1237/10000
1/1 [==============================] - 0s 4ms/step - loss: 100472.2734
Epoch 1238/10000
1/1 [==============================] - 0s 5ms/step - loss: 52513.0977
Epoch 1239/10000
1/1 [==============================] - 0s 4ms/step - loss: 64335.1562
Epoch 1240/10000
1/1 [==============================] - 0

1/1 [==============================] - 0s 4ms/step - loss: 87727.7188
Epoch 1324/10000
1/1 [==============================] - 0s 4ms/step - loss: 91858.2266
Epoch 1325/10000
1/1 [==============================] - 0s 4ms/step - loss: 88179.9375
Epoch 1326/10000
1/1 [==============================] - 0s 4ms/step - loss: 80683.5312
Epoch 1327/10000
1/1 [==============================] - 0s 4ms/step - loss: 100189.9922
Epoch 1328/10000
1/1 [==============================] - 0s 4ms/step - loss: 60857.2539
Epoch 1329/10000
1/1 [==============================] - 0s 5ms/step - loss: 88700.5312
Epoch 1330/10000
1/1 [==============================] - 0s 4ms/step - loss: 47013.4180
Epoch 1331/10000
1/1 [==============================] - 0s 4ms/step - loss: 77152.0078
Epoch 1332/10000
1/1 [==============================] - 0s 5ms/step - loss: 82165.3750
Epoch 1333/10000
1/1 [==============================] - 0s 4ms/step - loss: 70605.5234
Epoch 1334/10000
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 4ms/step - loss: 79194.7578
Epoch 1418/10000
1/1 [==============================] - 0s 4ms/step - loss: 117771.4375
Epoch 1419/10000
1/1 [==============================] - 0s 4ms/step - loss: 102649.9453
Epoch 1420/10000
1/1 [==============================] - 0s 4ms/step - loss: 97180.7891
Epoch 1421/10000
1/1 [==============================] - 0s 4ms/step - loss: 121555.0625
Epoch 1422/10000
1/1 [==============================] - 0s 4ms/step - loss: 70937.0703
Epoch 1423/10000
1/1 [==============================] - 0s 4ms/step - loss: 97967.4609
Epoch 1424/10000
1/1 [==============================] - 0s 4ms/step - loss: 98732.4141
Epoch 1425/10000
1/1 [==============================] - 0s 4ms/step - loss: 56396.5195
Epoch 1426/10000
1/1 [==============================] - 0s 4ms/step - loss: 120998.0547
Epoch 1427/10000
1/1 [==============================] - 0s 5ms/step - loss: 121774.0391
Epoch 1428/10000
1/1 [==============================] -

1/1 [==============================] - 0s 4ms/step - loss: 99513.4766
Epoch 1512/10000
1/1 [==============================] - 0s 4ms/step - loss: 98083.7891
Epoch 1513/10000
1/1 [==============================] - 0s 4ms/step - loss: 63905.7383
Epoch 1514/10000
1/1 [==============================] - 0s 4ms/step - loss: 50162.0039
Epoch 1515/10000
1/1 [==============================] - 0s 4ms/step - loss: 77687.9609
Epoch 1516/10000
1/1 [==============================] - 0s 4ms/step - loss: 244559.1250
Epoch 1517/10000
1/1 [==============================] - 0s 4ms/step - loss: 56364.4062
Epoch 1518/10000
1/1 [==============================] - 0s 4ms/step - loss: 94985.5312
Epoch 1519/10000
1/1 [==============================] - 0s 4ms/step - loss: 130835.3047
Epoch 1520/10000
1/1 [==============================] - 0s 4ms/step - loss: 108509.4375
Epoch 1521/10000
1/1 [==============================] - 0s 4ms/step - loss: 104836.8984
Epoch 1522/10000
1/1 [==============================] - 

In [22]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 584ms/step
[3862.535](test_target) - [3857.4526](prediction) = 5.0823632812498545


In [23]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.0013158102855378281

In [24]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [25]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Desemprego,Bahia - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Bahia - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.945479,-1.273847,-0.903379,-0.785203,-1.284368,-1.255052,-0.439210,-1.233495,-1.334114,1.148205,...,1.308949,1.412346,-1.275303,-1.412669,1.305056,-1.410138,1.251494,1.242982,-1.380085,1.414193
1,-0.438058,0.104961,-0.490611,-0.626022,0.129545,0.063049,1.383787,0.017693,0.260725,0.140893,...,-0.190804,-0.643250,0.108311,0.763554,-1.124355,0.612152,-0.055376,-0.037329,0.422604,-0.713648
2,1.383537,1.168887,1.393990,1.411224,1.154823,1.192003,-0.944578,1.215803,1.073389,-1.289098,...,-1.118144,-0.769096,1.166992,0.649115,-0.180701,0.797986,-1.196117,-1.205654,0.957481,-0.700545


train_target:


,Bahia - Consumo de Cimento (t)
0,2825.491
1,3349.109
2,3615.893


test_input:


...

,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Desemprego,Bahia - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Bahia - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.416125,-1.086745,-0.129344,-0.207609,0.000408,0.737609,0.749286,0.972824,-0.47095,1.108405,...,-1.482779,-2.260447,0.290535,1.940009,0.112434,-1.032368,-0.187335,-0.1229,0.783824,-1.669544


test_target:


,Bahia - Consumo de Cimento (t)
12,3862.535


1/1 [==============================] - 1s 563ms/step
Error: 5.0823632812498545




[3628.080810546875,
 3683.952392578125,
 3461.284912109375,
 3940.895751953125,
 3838.285400390625,
 3506.93896484375,
 3235.2021484375,
 3192.349609375,
 3244.73974609375,
 3857.45263671875]

In [26]:
display(targets)
display(predictions)

[3744.322,
 3932.905,
 4055.14,
 3732.551999999999,
 3424.967999999999,
 3063.3,
 2990.152,
 3245.585,
 3803.9,
 3862.535]

[3628.080810546875,
 3683.952392578125,
 3461.284912109375,
 3940.895751953125,
 3838.285400390625,
 3506.93896484375,
 3235.2021484375,
 3192.349609375,
 3244.73974609375,
 3857.45263671875]

In [27]:
mae = mean_absolute_error(predictions, targets)
mae

288.6877158203127

In [28]:
porcentage = mae/np.mean(targets)
porcentage

0.08051452387363148